In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install efficientnet-pytorch
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import albumentations
import os
from torch.utils.data import Dataset
import cv2
from torch.utils.data import DataLoader
import torchvision
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

In [3]:
PATH = "../input/plant-pathology-2021-fgvc8/train_images"
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
LEARNING_RATE = 3e-4
DEVICE = "cuda" if (torch.cuda.is_available()) else "cpu"
EPOCHS = 10

In [4]:
train_augmentation = albumentations.Compose([
    albumentations.Resize(WIDTH, HEIGHT),
])

val_augmentation = albumentations.Compose([
    albumentations.Resize(WIDTH, HEIGHT),
])

In [5]:
os.listdir("../input/plant-pathology-2021-fgvc8")

In [6]:
df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")

In [7]:
df.head()

In [8]:
df.labels.unique()

In [9]:
df.labels.value_counts()

In [10]:
df.size

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_df, val_df = train_test_split(df,test_size = 0.3, random_state = 42 , stratify = df.labels)

In [13]:
val_df.reset_index(inplace = True)  
train_df.reset_index(inplace = True)

In [14]:
val_df.drop("index", axis = 1, inplace = True)
train_df.drop("index", axis = 1, inplace = True)

In [15]:
from sklearn.preprocessing import OneHotEncoder
oneHot = OneHotEncoder()
oneHot.fit(train_df[["labels"]])
categorical = oneHot.transform(train_df[["labels"]]).toarray()
temp = pd.DataFrame(categorical,columns =oneHot.categories_[0])
train_df.drop("labels",inplace = True,axis = 1)
train_df = pd.concat([train_df,temp],axis = 1)

In [16]:
oneHot1 = OneHotEncoder()
oneHot1.fit(val_df[["labels"]])
categorical = oneHot1.transform(val_df[["labels"]]).toarray()
temp = pd.DataFrame(categorical,columns =oneHot1.categories_[0])
val_df.drop("labels",inplace = True,axis = 1)
val_df = pd.concat([val_df,temp],axis = 1)

In [17]:
train_df.head()

In [18]:
val_df.head()

In [19]:
train_df.shape

In [20]:
val_df.shape

In [21]:
class LeafDataset(Dataset):
    
    def __init__(self,df,path,augmentation = None):
        self.df = df
        self.path = path
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        image = cv2.imread(os.path.join(self.path,self.df["image"][index]))
        target = torch.tensor((self.df.iloc[0].values[1:]).astype("float"))
        if self.augmentation:
            image = self.augmentation(image = image)["image"]
        image = torch.tensor(image/255.0)
        image =image.permute(2,0,1)
        return image.to(device = DEVICE),target.to(device = DEVICE)

In [22]:
train_dataset = LeafDataset(train_df,PATH,train_augmentation)
val_dataset = LeafDataset(val_df,PATH,val_augmentation)

In [23]:
(train_dataset[0][0]).shape

In [24]:
(train_dataset[0][1]).shape

In [25]:
# plt.imshow(my_dataset[0][0].permute(1,2,0))

In [26]:
train_loader = DataLoader(train_dataset,shuffle = True, batch_size = BATCH_SIZE)
val_loader = DataLoader(val_dataset,shuffle = True, batch_size = BATCH_SIZE)

In [27]:
temp = next(iter(train_loader))

In [28]:
temp[0][0].shape

In [29]:
plt.imshow(temp[0][0].permute(1,2,0).cpu())

In [30]:
class MyModel(nn.Module):
    
    def __init__(self , output_dims):
        super().__init__()
        self.model = EfficientNet.from_pretrained("efficientnet-b0")
        self.out = nn.Linear(1280,output_dims)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self,x):
        z = self.model.extract_features(x)
        z = nn.functional.adaptive_avg_pool2d(z,1)
        z = torch.flatten(z,1)
        z = self.out(self.dropout(z))
        return z

In [31]:
model = MyModel(len(df.labels.unique()))

In [32]:
model(torch.rand([1,3,256,256])).shape

In [33]:
model.to(device = DEVICE)
model.double()

In [34]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = LEARNING_RATE)

In [35]:
def training(x,y):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction , y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [36]:
@torch.no_grad()
def accuracy(x,y):
    model.eval()
    prediction = model(x)
    is_correct = ( (prediction>0.5) == y)
    return is_correct.cpu().numpy()

In [37]:
@torch.no_grad()
def loss_validation(x,y):
    model.eval()
    prediction = model(x)
    loss = loss_fn(prediction , y)
    return loss.item()

In [43]:
loss_train = []
loss_val = []
accuracy_train = []
accuracy_val = []

def train():
    for epoch in range(EPOCHS):
        loss_train_epoch = [] 
        loss_val_epoch = []
        accuracy_train_epoch = []
        accuracy_val_epoch = []
        for batch in tqdm(train_loader,position=0, leave=True):
            x,y = batch
            loss = training(x,y)
            acc = accuracy(x,y).sum()
            loss_train_epoch.append(loss)
            accuracy_train_epoch.append(acc)
            
        for index,batch in enumerate(val_loader):
            x,y = batch
            loss = loss_validation(x,y)
            acc = accuracy(x,y).sum()
            loss_val_epoch.append(loss)
            accuracy_val_epoch.append(acc)
            
        loss_train.append(sum(loss_train_epoch))
        loss_val.append(sum(loss_val_epoch))
        accuracy_train.append(sum(accuracy_train_epoch))
        accuracy_val.append(sum(accuracy_val_epoch))
        
    

In [44]:
train()

In [45]:
loss_train